In [1]:
# Import libraries
import pandas as pd
import numpy as np
import panel as pn
from panel.interact import interact
pn.extension()

import hvplot.pandas

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [2]:
# Create DataFrame from csv file
df = pd.read_csv("dkb_2023.csv", sep=";", encoding = 'ISO-8859-1', skiprows=6)

df = df.drop(["Wertstellung","Buchungstext","Gläubiger-ID","BLZ","Kontonummer", 
              "Verwendungszweck","Mandatsreferenz","Kundenreferenz","Unnamed: 11"],axis=1)

# Rename columns
df = df.rename(columns={'Auftraggeber / Begünstigter':'Auftraggeber'})
df = df.rename(columns={'Betrag (EUR)':'Betrag'})

# Insert additional columns
df['Kategorie'] = 'Sonstiges'
df['Monat'] = 'Nicht definiert'
df['Jahr'] = 'Nicht definiert'

# Format Betrag
df['Betrag'] = df['Betrag'].str.replace(r'\.', '', regex=True)
df['Betrag'] = df['Betrag'].str.replace(r'\,', '.', regex=True)
df['Betrag'] = df['Betrag'].astype(float)

# Format Buchungstag
df['Buchungstag'] = df['Buchungstag'].astype(str)

# Get year and month
df['Jahr'] = df['Buchungstag'].str.slice(6,10)
df['Monat'] = df['Buchungstag'].str.slice(3,5) 
df['Monat'] = df['Monat'].str.lstrip('0')


In [3]:
# Categorization
# Lebensmittel
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'SPAR|Sutterluety|Vorarlberg Milch'),'Lebensmittel', df['Kategorie'])

# Mittagessen
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'Kommod'),'Mittagessen', df['Kategorie'])

# Jause
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'Manga|MANGOLD'),'Jause', df['Kategorie'])

# Handy
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'A1 Telekom'),'Handy', df['Kategorie'])

# Skifahren
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'Bergbahn'),'Skifahren', df['Kategorie'])

# Investments
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'FLATEX'),'Investments', df['Kategorie'])

# Drogerie
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'DM-'),'Drogerie', df['Kategorie'])

# Krankenversicherung
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'GENERALI'),'Krankenversicherung', df['Kategorie'])

# Amazon
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'AMZN'),'Amazon', df['Kategorie'])

# Klamotten
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'Facona|Garzon|HM'),'Klamotten', df['Kategorie'])

# Gehalt
df['Kategorie'] = np.where(df['Auftraggeber'].str.contains(
    'Pantec'),'Gehalt', df['Kategorie'])

In [4]:
# Panel - expenses chart with dropdown filters

summe = pn.widgets.TextInput(name="Summe", value="0")
select_year = pn.widgets.Select(name='Jahr auswählen', options=['2023'])
select_month = pn.widgets.Select(name='Monat auswählen', options=['Alle', 'Januar', 'Februar', 'März', 'April'])
select_category = pn.widgets.Select(name='Kategorie auswählen', 
                    options=['Alle','Lebensmittel','Mittagessen','Jause','Handy','Skifahren','Drogerie','Krankenversicherung',
                             'Amazon','Klamotten'])

# Callback function
@pn.depends(select_year.param.value, select_month.param.value, select_category.param.value)
def update_plot(year, month, category):
    
    if month == 'Januar': month_nr = '1'
    elif month == 'Februar': month_nr = '2'
    elif month == 'März': month_nr = '3'
    elif month == 'April': month_nr = '4'
    
    # Expenses year total
    if month == 'Alle' and category == 'Alle':
        
        data = df[(df['Jahr'] == year)]
        data = df.groupby('Kategorie')['Betrag'].sum().reset_index()
        data = data[data['Kategorie'].str.contains("Gehalt|Investments") == False]
        data['Betrag'] = data['Betrag'].abs()
        data = data.sort_values(by='Betrag', ascending=True)    
        plot = data.hvplot.barh(x='Kategorie', y='Betrag', xlabel='', ylabel='', title="Ausgaben Jahr", height=400, grid=True)

    # Expenses month total
    elif month != 'Alle' and category == 'Alle':
        
        data = df[(df['Monat'] == month_nr) & (df['Jahr'] == year)]
        data = data.groupby('Kategorie')['Betrag'].sum().reset_index()
        data = data[data['Kategorie'].str.contains("Gehalt|Investments") == False]
        data['Betrag'] = data['Betrag'].abs()
        data = data.sort_values(by='Betrag', ascending=True)
        plot = data.hvplot.barh(x='Kategorie', y='Betrag', xlabel='', ylabel='', title="Ausgaben Monat", height=400, grid=True)
    
    # Expenses category total
    elif month == 'Alle' and category != 'Alle':

        data = df[(df['Kategorie'] == category) & (df['Jahr'] == year)]
        data = data.groupby('Monat')['Betrag'].sum().reset_index()
        data['Betrag'] = data['Betrag'].abs()
        data['Monat'] = np.where(data['Monat'] == '1','Januar', data['Monat'])
        data['Monat'] = np.where(data['Monat'] == '2','Februar', data['Monat'])       
        data['Monat'] = np.where(data['Monat'] == '3','März', data['Monat'])
        data['Monat'] = np.where(data['Monat'] == '4','April', data['Monat'])
        data['Monat'] = np.where(data['Monat'] == '5','Mai', data['Monat'])
        data['Monat'] = np.where(data['Monat'] == '6','Juni', data['Monat']) 
        data['Monat'] = np.where(data['Monat'] == '7','Juli', data['Monat'])
        data['Monat'] = np.where(data['Monat'] == '8','August', data['Monat']) 
        data['Monat'] = np.where(data['Monat'] == '9','September', data['Monat'])
        data['Monat'] = np.where(data['Monat'] == '10','Oktober', data['Monat'])        
        data['Monat'] = np.where(data['Monat'] == '11','November', data['Monat'])
        data['Monat'] = np.where(data['Monat'] == '12','Dezember', data['Monat'])
        plot = data.hvplot.bar(x='Monat', y='Betrag', xlabel='', ylabel='', title="Jahresausgabe Kategorie", height=400, grid=True)
  
    # Expenses category month
    elif month != 'Alle' and category != 'Alle':
        
        data = df[(df['Monat'] == month_nr) & (df['Kategorie'] == category) & (df['Jahr'] == year)]
        data = data.groupby('Kategorie')['Betrag'].sum().reset_index()
        data = data[data['Kategorie'].str.contains("Gehalt|Investments") == False]
        data['Betrag'] = data['Betrag'].abs()
        data = data.sort_values(by='Betrag', ascending=True)
        plot = data.hvplot.barh(x='Kategorie', y='Betrag', xlabel='', ylabel='', title="Monatsausgabe Kategorie", height=400, grid=True)
        
    my_sum = data['Betrag'].sum()
    my_sum = int(my_sum)
    summe.value = str(my_sum) + " €"
    
    return plot

expensesChart = pn.Row(update_plot, pn.Column(select_year, select_month, select_category, summe))
expensesChart

Row
    [0] ParamFunction(function, _pane=HoloViews)
    [1] Column
        [0] Select(name='Jahr auswählen', options=['2023'], value='2023')
        [1] Select(name='Monat auswählen', options=['Alle', 'Januar', ...], value='Alle')
        [2] Select(name='Kategorie auswählen', options=['Alle', 'Lebensmittel', ...], value='Alle')
        [3] TextInput(name='Summe', value='10296 €')

In [5]:
# Dashboard

template = pn.template.FastListTemplate(
    title="Finanz Dashboard",
    sidebar=[
        pn.pane.Markdown("## *Ein- und Ausgänge sind gesondert zu betrachten! <br><br> Julia Beer, 08.04.2023*"),
        pn.pane.JPG('Julia3.jpg', sizing_mode='scale_width') 
    ],
    main=[
        pn.Row(expensesChart, height=400)
    ]
)

"""         )
        pn.GridBox(
            total_chart,
            total_table,
            ncols=2,
            width = 600,
            sizing_mode = 'stretch_width'
        )        
    ]
) """

template.show()

Launching server at http://localhost:53395


# Notizen

**Virtuelle Umgebung aktivieren und Jupyter Notebook ausführen:**
1) Mit cd auf Projektverzeichnis navigieren: C:\Users\Tobias\Documents\Projekte\PythonTest
2) .\.venv\Scripts\activate ausführen
3) jupyter lab ausführen

**Virtuelle Umgebung deaktivieren:**  
Mit cd auf Projektverzeichnis navigieren und deactivate ausführen

# Youtube
How to Create a Beautiful Python Visualization Dashboard With Panel/Hvplot

# Todo:
* vsCode einrichten
* Kategorisierung verbessern
* Neue Spalte für Unterscheidung Ein/Ausgänge
* Grafik für Ausgaben erstellen